# Financial Data Copilot - Exploratory Analysis

This notebook provides exploratory data analysis for the Financial Data Copilot project.

In [ ]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns

# Load environment variables
load_dotenv()

# Database connection
conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    database=os.getenv("DB_NAME", "llm_finance"),
    user=os.getenv("DB_USER", "postgres"),
    password=os.getenv("DB_PASSWORD", "postgres"),
    port=os.getenv("DB_PORT", "5432")
)

# Create a cursor
cursor = conn.cursor()

## 1. Companies Overview

In [ ]:
# Load companies data
companies_df = pd.read_sql_query("SELECT * FROM finance.companies", conn)
companies_df.head()

In [ ]:
# Sector distribution
plt.figure(figsize=(10, 6))
sector_counts = companies_df['sector'].value_counts()
sns.barplot(x=sector_counts.values, y=sector_counts.index)
plt.title('Company Distribution by Sector')
plt.xlabel('Number of Companies')
plt.ylabel('Sector')
plt.show()

## 2. Financial Statements Analysis

In [ ]:
# Load financial statements data
financial_df = pd.read_sql_query("""
    SELECT fs.*, c.symbol, c.name 
    FROM finance.financial_statements fs
    JOIN finance.companies c ON fs.company_id = c.id
""", conn)
financial_df.head()

In [ ]:
# Extract financial metrics from JSON data
financial_df['revenue'] = financial_df['data'].apply(lambda x: x.get('revenue') if x else None)
financial_df['net_income'] = financial_df['data'].apply(lambda x: x.get('net_income') if x else None)
financial_df['total_assets'] = financial_df['data'].apply(lambda x: x.get('total_assets') if x else None)
financial_df['total_liabilities'] = financial_df['data'].apply(lambda x: x.get('total_liabilities') if x else None)
financial_df['shareholders_equity'] = financial_df['data'].apply(lambda x: x.get('shareholders_equity') if x else None)

In [ ]:
# Revenue trends by company
revenue_df = financial_df[financial_df['statement_type'] == 'income_statement'][['symbol', 'fiscal_year', 'revenue']].dropna()
revenue_pivot = revenue_df.pivot(index='fiscal_year', columns='symbol', values='revenue')

plt.figure(figsize=(12, 8))
revenue_pivot.plot(kind='line')
plt.title('Revenue Trends by Company')
plt.xlabel('Fiscal Year')
plt.ylabel('Revenue')
plt.legend(title='Company')
plt.show()

## 3. Stock Price Analysis

In [ ]:
# Load stock prices data
stock_df = pd.read_sql_query("""
    SELECT sp.*, c.symbol, c.name 
    FROM finance.stock_prices sp
    JOIN finance.companies c ON sp.company_id = c.id
""", conn)
stock_df.head()

In [ ]:
# Stock price trends
stock_df['date'] = pd.to_datetime(stock_df['date'])
recent_stock_df = stock_df[stock_df['date'] >= '2020-01-01']

plt.figure(figsize=(15, 10))
for symbol in recent_stock_df['symbol'].unique()[:5]:  # Limit to first 5 companies for clarity
    symbol_data = recent_stock_df[recent_stock_df['symbol'] == symbol]
    plt.plot(symbol_data['date'], symbol_data['close_price'], label=symbol)
    
plt.title('Stock Price Trends')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend()
plt.show()

## 4. Document Analysis

In [ ]:
# Load documents data
documents_df = pd.read_sql_query("SELECT * FROM finance.documents", conn)
documents_df.head()

In [ ]:
# Document type distribution
plt.figure(figsize=(8, 6))
doc_type_counts = documents_df['document_type'].value_counts()
sns.barplot(x=doc_type_counts.values, y=doc_type_counts.index)
plt.title('Document Distribution by Type')
plt.xlabel('Number of Documents')
plt.ylabel('Document Type')
plt.show()

## 5. Close Connection

In [ ]:
# Close database connection
cursor.close()
conn.close()